To run the Ascend TVM on Jupyter, you need to set some environment first

In [15]:
# Ps. Using Mind Studio via X11 is like eating shi*t

In [25]:
import os

os.environ['PYTHONPATH'] = '/home/team1/.mindstudio/huawei/ddk/1.60.T33.0.B162/ddk/site-packages/te.egg:'+\
    '/home/team1/.mindstudio/huawei/ddk/1.60.T33.0.B162/ddk/site-packages/topi.egg:'+\
    '/home/team1/1.60/MindStudio-ubuntu/tools/tbe_tools/py_utils/'

os.environ['LD_LIBRARY_PATH'] = '/home/team1/.mindstudio/huawei/ddk/1.60.T33.0.B162/ddk/lib/x86_64-linux-gcc5.4/'

os.environ['PATH'] += ':/home/team1/.mindstudio/huawei/ddk/1.60.T33.0.B162/ddk/toolchains/ccec-linux/bin/'

Import Libs

In [7]:
from te import tvm
from te.platform import cce_params as cce
from te.platform.cce_build import build_config

Define sqrt module

In [8]:
shape = [512, 1024]
dtype = 'float16'
power_num = tvm.const(0.5, dtype = dtype)
data = tvm.placeholder(shape, name="data", dtype=dtype)
vlog_t = tvm.compute(shape, lambda *indice: tvm.log(data(*indice)), name = "vlog_t")
vmuls_t = tvm.compute(shape, lambda *indice: vlog_t(*indice) * power_num, name = "vmuls_t")
vexp_t = tvm.compute(shape, lambda *indice: tvm.exp(vmuls_t(*indice)), name = "vexp_t")
s = tvm.create_schedule(vexp_t.op)

Print IR

In [11]:
# print IR, option
with build_config:
    print(tvm.lower(s, [data, vexp_t], simple_mode=True))

// attr [0] pragma_ = "device"
// attr [vlog_t] storage_scope = "global"
allocate vlog_t[float16 * 512 * 1024]
// attr [vmuls_t] storage_scope = "global"
allocate vmuls_t[float16 * 512 * 1024]
produce vlog_t {
  for (i0, 0, 512) {
    for (i1, 0, 1024) {
      vlog_t[((i0*1024) + i1)] = log(data[((i0*1024) + i1)])
    }
  }
}
produce vmuls_t {
  for (i0, 0, 512) {
    for (i1, 0, 1024) {
      vmuls_t[((i0*1024) + i1)] = (vlog_t[((i0*1024) + i1)]*0.500000h)
    }
  }
}
produce vexp_t {
  for (i0, 0, 512) {
    for (i1, 0, 1024) {
      vexp_t[((i0*1024) + i1)] = exp(vmuls_t[((i0*1024) + i1)])
    }
  }
}



To Optimize the Computation Schedule

In [14]:
scope_ubuf = cce.scope_ubuf
# Caching
dataL = s.cache_read(data, scope_ubuf, [vlog_t])
vlog_tL = s.cache_write(vlog_t, scope_ubuf)
vmuls_tL = s.cache_write(vmuls_t, scope_ubuf)
vexp_tL = s.cache_write(vexp_t, scope_ubuf)

In [16]:
# remove computation redundance
s[vlog_t].compute_inline()
s[vmuls_t].compute_inline()

In [19]:
# Data spliting & tiling
factor = 32
xo, xi = s[vexp_t].split(vexp_t.op.axis[0], factor)     #xo=16, xi=32
eo, ei = s[vexp_tL].split(vexp_tL.op.axis[0], factor)   #eo=16, ei=32
mo, mi = s[vmuls_tL].split(vmuls_tL.op.axis[0], factor) #mo=16. mi=32
lo, li = s[vlog_tL].split(vlog_tL.op.axis[0], factor)   #lo=16, li=32
do, di = s[dataL].split(dataL.op.axis[0], factor)       #do=16, di=32

In [20]:
# Ops Fusion
s[vexp_tL].compute_at(s[vexp_t], xo)     #将vexp_tL语句块聚合到vexp_t的最外层轴上
s[vmuls_tL].compute_at(s[vexp_t], xo)    #将vmuls_tL语句块聚合到vexp_t的最外层轴上
s[vlog_tL].compute_at(s[vexp_t], xo)     #将vlog_tL语句块聚合到vexp_t的最外层轴上
s[dataL].compute_at(s[vexp_t], xo)       #将dataL语句块聚合到vexp_t的最外层轴上

In [21]:
# Instruction Mapping
s[vlog_tL].emit_insn(li, 'vector_ln')        #替换的轴为li=32,为最外层轴，即vlog_tL的整个代码块进行替换
s[vmuls_tL].emit_insn(mi, 'vector_muls')     #替换的轴为mi=32,为最外层轴，即vmuls_tL的整个代码块进行替换
s[vexp_tL].emit_insn(ei, 'vector_exp')       #替换的轴为ei=32,为最外层轴，即vexp_tL的整个代码块进行替换
s[dataL].emit_insn(di, 'dma_copy')           #替换的轴为di=32,为最外层轴，即dataL的整个代码块进行替换
s[vexp_t].emit_insn(xi, 'dma_copy')          #替换的轴为xi=32,为最外层轴，即vexp_t的整个代码块进行替换

Print Optimized IR

In [22]:
with build_config:
    print(tvm.lower(s, [data, vexp_t], simple_mode=True))

// attr [0] pragma_ = "device"
// attr [data.local.UB] storage_scope = "local.UB"
allocate data.local.UB[float16 * 32 * 1024]
 custom_new { 0 }
 custom_delete { nop(<args>); }
// attr [vlog_t.local.UB] storage_scope = "local.UB"
allocate vlog_t.local.UB[float16 * 32 * 1024]
 custom_new { 65536 }
 custom_delete { nop(<args>); }
// attr [vmuls_t.local.UB] storage_scope = "local.UB"
allocate vmuls_t.local.UB[float16 * 32 * 1024]
 custom_new { 131072 }
 custom_delete { nop(<args>); }
produce vexp_t {
  // attr [iter_var(cce, , cce)] coproc_scope = 2
  cce.coproc_dep_push(6, 5, 0)
  set_vector_mask((uint64)18446744073709551615, (uint64)18446744073709551615)
  for (i0.outer, 0, 16) {
    produce data.local.UB {
      // attr [iter_var(cce, , cce)] coproc_scope = 5
      cce.coproc_dep_pop(6, 5, 0)
      copy_gm_to_ubuf(tvm_address_of(data.local.UB[0]), tvm_address_of(data[(i0.outer*32768)]), 0, 1, 2048, 0, 0)
      cce.coproc_dep_push(5, 2, 0)
    }
    produce vlog_t.local.UB {
      // att

Build the Operator

In [31]:
with build_config:
    tvm.build(s, [data, vexp_t], "cce", name="sqrt_demo")

TBE算子编译之后，会在当前目录下的kernel_meta目录下生成如下内容：
* 算子的二进制文件*.o，文件以tvm.build中的name参数命名。
* 算子的信息描述文件*.json，文件以tvm.build中的name参数命名。

In [33]:
ls kernel_meta/

sqrt_demo.json  sqrt_demo.o
